In [1]:
import logging
import os
import sys
import copy
import re

import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
from typing import List, Callable, NoReturn, NewType, Any
import dataclasses
from datasets import load_metric, load_from_disk, Dataset, DatasetDict

from konlpy.tag import Mecab
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer
from transformers import (
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

In [2]:
import pandas as pd

In [2]:
sys.path.append('../')

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

from utils_qa import postprocess_qa_predictions, check_no_error
from trainer_qa import QuestionAnsweringTrainer
from retrieval import SparseRetrieval

from arguments import (
    ModelArguments,
    DataTrainingArguments,
)

## Raw Data

In [10]:
datasets = load_from_disk('/opt/ml/data/train_dataset')
datasets

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [5]:
train_data = datasets['train']

train_size = len(train_data)
print('Size of Train Data: %d' %train_size)

Size of Train Data: 3952


## Data Element

In [6]:
train_data[13]

{'title': '장면',
 'context': '장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로 취임하였다. 귀국 무렵 대한민국 국회와 이승만은 갈등하고 있었다. 그는 중간에서 양자의 조정 역할을 하려 했으나 실패하였다 그는 귀국 즉시 총리직에 취임하지 않고, 1주일 간의 여유를 얻어 요인들을 만나 의논해 보았다 그는 “일반적으로 이 박사에 대한 평이 좋지 않아 총리직을 맡을 생각이 간절하지 않았으나 이왕 인준도 받았으니 하는 데까지 하다가 할 수 없으면 그만두라”라는 주변의 의견대로 총리직을 맡게 되었다 당초 이승만은 장면에게 인사권을 위임하겠다고 하였으나, “5석 중 3석을 총리가 정하였으니 내무에는 이순용(李淳鎔), 국방에는 이기붕을 써 주시오”하고 종용하자, 당초 약속과는 달랐으나 장면은 받아들였다 국무총리 재임 중 그는 호화로운 식단을 기피하고 직접 도시락을 싸들고 출퇴근하였다.\\n\\n \\n8월에는 자유당이 창당되자 자유당에 입당했다. 12월에는 원외 자유당이 결성되면서 장면이 속한 자유당은 원내 자유당으로 불리게 되었다. 원내 자유당에서는 1952년 제2대 정부통령 선거에서 그를 대통령 후보로 추대하려는 일파와, 내각 책임제로 개헌하여 대통령직을 유명무실화하게 한 뒤 그를 내각 책임제 하의 국무총리로 추대하려는 세력이 나타났다. 흥사단도 장면을 지지하였다. 이후 흥사단의 지지를 이유로 장면은 흥사단계로 분류되었으나 장면은 이를 부정하였다. 장면을 대통령 후보로 추대하려는 세력과 내각책임제 개헌 후 장면을 총리로 추대하려는 세력의 존재가 이승만의 귀에 들어가면서 장면은 정치적으로 궁지에 몰리게 된다.',
 'question': '장면이 한국으로 돌아온 년도는?',
 'id': 'mrc-0-001373',
 'answers': {'answer_start': [26], 'text': ['1951년']},
 'document_id': 6930,
 '__index_level_0__': 899}

## Aumentation

In [7]:
def left_augmentation(data, k=12) :
    assert isinstance(data, dict)
    
    context = data['context']
    answer = data['answers']
    answer_pos, answer_text = answer['answer_start'][0], answer['text'][0]

    answer_pos_left = answer_pos - k
    while context[answer_pos_left] != ' ' :
        answer_pos_left -= 1
    answer_pos_left += 1
    answer_text_left = context[answer_pos_left:answer_pos+len(answer_text)] 

    data_left = copy.deepcopy(data)
    data_left['id'] = data['id'] + '_0'
    data_left['answers'] = {'answer_start' : [answer_pos_left], 'text' : [answer_text_left]}
    return data_left


In [11]:
print(train_data[13]['answers'])

{'answer_start': [26], 'text': ['1951년']}


In [8]:
left_augmentation(train_data[13])

{'title': '장면',
 'context': '장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로 취임하였다. 귀국 무렵 대한민국 국회와 이승만은 갈등하고 있었다. 그는 중간에서 양자의 조정 역할을 하려 했으나 실패하였다 그는 귀국 즉시 총리직에 취임하지 않고, 1주일 간의 여유를 얻어 요인들을 만나 의논해 보았다 그는 “일반적으로 이 박사에 대한 평이 좋지 않아 총리직을 맡을 생각이 간절하지 않았으나 이왕 인준도 받았으니 하는 데까지 하다가 할 수 없으면 그만두라”라는 주변의 의견대로 총리직을 맡게 되었다 당초 이승만은 장면에게 인사권을 위임하겠다고 하였으나, “5석 중 3석을 총리가 정하였으니 내무에는 이순용(李淳鎔), 국방에는 이기붕을 써 주시오”하고 종용하자, 당초 약속과는 달랐으나 장면은 받아들였다 국무총리 재임 중 그는 호화로운 식단을 기피하고 직접 도시락을 싸들고 출퇴근하였다.\\n\\n \\n8월에는 자유당이 창당되자 자유당에 입당했다. 12월에는 원외 자유당이 결성되면서 장면이 속한 자유당은 원내 자유당으로 불리게 되었다. 원내 자유당에서는 1952년 제2대 정부통령 선거에서 그를 대통령 후보로 추대하려는 일파와, 내각 책임제로 개헌하여 대통령직을 유명무실화하게 한 뒤 그를 내각 책임제 하의 국무총리로 추대하려는 세력이 나타났다. 흥사단도 장면을 지지하였다. 이후 흥사단의 지지를 이유로 장면은 흥사단계로 분류되었으나 장면은 이를 부정하였다. 장면을 대통령 후보로 추대하려는 세력과 내각책임제 개헌 후 장면을 총리로 추대하려는 세력의 존재가 이승만의 귀에 들어가면서 장면은 정치적으로 궁지에 몰리게 된다.',
 'question': '장면이 한국으로 돌아온 년도는?',
 'id': 'mrc-0-001373_0',
 'answers': {'answer_start': [14], 'text': ['임명되었으나 이듬해인 1951년']},
 'document_id': 6930,
 '__index_level_0__': 89

In [15]:
def right_augmentation(data) :
    context = data['context']
    answer = data['answers']
    answer_pos, answer_text = answer['answer_start'][0], answer['text'][0]

    answer_pos_right = min(len(context)-1,answer_pos + len(answer_text) + np.random.randint(12,24))
    while context[answer_pos_right] != ' ' and answer_pos_right < len(context)-1:
        answer_pos_right += 1
    answer_text_right = context[answer_pos:answer_pos_right+1] 

    data_right = copy.deepcopy(data)
    data_right['id'] = data['id'] + '_1'
    data_right['answers'] = {'answer_start' : [answer_pos], 'text' : [answer_text_right]}
    return data_right

In [13]:
print(train_data[13]['answers'])

{'answer_start': [26], 'text': ['1951년']}


In [16]:
right_augmentation(train_data[13])

{'title': '장면',
 'context': '장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로 취임하였다. 귀국 무렵 대한민국 국회와 이승만은 갈등하고 있었다. 그는 중간에서 양자의 조정 역할을 하려 했으나 실패하였다 그는 귀국 즉시 총리직에 취임하지 않고, 1주일 간의 여유를 얻어 요인들을 만나 의논해 보았다 그는 “일반적으로 이 박사에 대한 평이 좋지 않아 총리직을 맡을 생각이 간절하지 않았으나 이왕 인준도 받았으니 하는 데까지 하다가 할 수 없으면 그만두라”라는 주변의 의견대로 총리직을 맡게 되었다 당초 이승만은 장면에게 인사권을 위임하겠다고 하였으나, “5석 중 3석을 총리가 정하였으니 내무에는 이순용(李淳鎔), 국방에는 이기붕을 써 주시오”하고 종용하자, 당초 약속과는 달랐으나 장면은 받아들였다 국무총리 재임 중 그는 호화로운 식단을 기피하고 직접 도시락을 싸들고 출퇴근하였다.\\n\\n \\n8월에는 자유당이 창당되자 자유당에 입당했다. 12월에는 원외 자유당이 결성되면서 장면이 속한 자유당은 원내 자유당으로 불리게 되었다. 원내 자유당에서는 1952년 제2대 정부통령 선거에서 그를 대통령 후보로 추대하려는 일파와, 내각 책임제로 개헌하여 대통령직을 유명무실화하게 한 뒤 그를 내각 책임제 하의 국무총리로 추대하려는 세력이 나타났다. 흥사단도 장면을 지지하였다. 이후 흥사단의 지지를 이유로 장면은 흥사단계로 분류되었으나 장면은 이를 부정하였다. 장면을 대통령 후보로 추대하려는 세력과 내각책임제 개헌 후 장면을 총리로 추대하려는 세력의 존재가 이승만의 귀에 들어가면서 장면은 정치적으로 궁지에 몰리게 된다.',
 'question': '장면이 한국으로 돌아온 년도는?',
 'id': 'mrc-0-001373_1',
 'answers': {'answer_start': [26], 'text': ['1951년 2월에 귀국한 후 정식으로 ']},
 'document_id': 6930,
 '__index_level_0__'

In [17]:
def mid_augmentation(data, k=12) :
    assert isinstance(data, dict)
    
    context = data['context']
    answer = data['answers']
    answer_pos = answer['answer_start'][0]

    answer_pos_left = answer_pos - int(k/2)
    while context[answer_pos_left] != ' ' :
        answer_pos_left -= 1
    answer_pos_left += 1

    answer_pos_right = answer_pos + int(k/2)
    while context[answer_pos_right] != ' ' :
        answer_pos_right += 1

    answer_text = context[answer_pos_left:answer_pos_right] 
    data_mid = copy.deepcopy(data)
    data_mid['id'] = data['id'] + '_2'
    data_mid['answers'] = {'answer_start' : [answer_pos], 'text' : [answer_text]}
    return data_mid


In [21]:
print(train_data['answers'][13])

{'answer_start': [26], 'text': ['1951년']}


In [22]:
mid_augmentation(train_data[13])

{'title': '장면',
 'context': '장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로 취임하였다. 귀국 무렵 대한민국 국회와 이승만은 갈등하고 있었다. 그는 중간에서 양자의 조정 역할을 하려 했으나 실패하였다 그는 귀국 즉시 총리직에 취임하지 않고, 1주일 간의 여유를 얻어 요인들을 만나 의논해 보았다 그는 “일반적으로 이 박사에 대한 평이 좋지 않아 총리직을 맡을 생각이 간절하지 않았으나 이왕 인준도 받았으니 하는 데까지 하다가 할 수 없으면 그만두라”라는 주변의 의견대로 총리직을 맡게 되었다 당초 이승만은 장면에게 인사권을 위임하겠다고 하였으나, “5석 중 3석을 총리가 정하였으니 내무에는 이순용(李淳鎔), 국방에는 이기붕을 써 주시오”하고 종용하자, 당초 약속과는 달랐으나 장면은 받아들였다 국무총리 재임 중 그는 호화로운 식단을 기피하고 직접 도시락을 싸들고 출퇴근하였다.\\n\\n \\n8월에는 자유당이 창당되자 자유당에 입당했다. 12월에는 원외 자유당이 결성되면서 장면이 속한 자유당은 원내 자유당으로 불리게 되었다. 원내 자유당에서는 1952년 제2대 정부통령 선거에서 그를 대통령 후보로 추대하려는 일파와, 내각 책임제로 개헌하여 대통령직을 유명무실화하게 한 뒤 그를 내각 책임제 하의 국무총리로 추대하려는 세력이 나타났다. 흥사단도 장면을 지지하였다. 이후 흥사단의 지지를 이유로 장면은 흥사단계로 분류되었으나 장면은 이를 부정하였다. 장면을 대통령 후보로 추대하려는 세력과 내각책임제 개헌 후 장면을 총리로 추대하려는 세력의 존재가 이승만의 귀에 들어가면서 장면은 정치적으로 궁지에 몰리게 된다.',
 'question': '장면이 한국으로 돌아온 년도는?',
 'id': 'mrc-0-001373_2',
 'answers': {'answer_start': [26], 'text': ['이듬해인 1951년 2월에']},
 'document_id': 6930,
 '__index_level_0__': 899}

In [23]:
class SpanAugmentation :
    def __init__(self, n=4, p=0.8, max_len=24, min_len=6) :
        self.n = n
        self.p = p
        self.max_len = max_len
        self.min_len = min_len

    def __call__(self, dataset) :
        assert isinstance(train_data, Dataset)
        data_list = [data for data in dataset]
        data_size = len(data_list)

        data_augmented = []
        rand_prob = np.random.rand(data_size)
        for i, data in enumerate(data_list) :
            if rand_prob[i] > self.p :
                continue
            for j in range(self.n) :
                direction = np.random.randint(3)
                if direction == 0 : # left
                    data_augmented.append(self.left_augmentation(data,j))
                elif direction == 1 : # right
                    data_augmented.append(self.right_augmentation(data,j))
                else : # mid
                    data_augmented.append(self.mid_augmentation(data,j))

        data_augmented = self.convert_to_dict(data_augmented)
        data_augmented = Dataset.from_dict(data_augmented)
        return data_augmented

    def convert_to_dict(self, data_list) :
        data_dict = {}
        for key in data_list[0].keys() :
            val_list = [data[key] for data in data_list]
            data_dict[key] = val_list
        return data_dict
    
    def left_augmentation(self, data, j) :
        context = data['context']
        answer = data['answers']
        answer_pos, answer_text = answer['answer_start'][0], answer['text'][0]

        answer_pos_left = max(0,answer_pos - np.random.randint(self.min_len, self.max_len))
        while context[answer_pos_left] != ' ' and answer_pos_left >= 0:
            answer_pos_left -= 1
        answer_pos_left += 1
        answer_text_left = context[answer_pos_left:answer_pos+len(answer_text)] 

        data_left = copy.deepcopy(data)
        data_left['id'] = data['id'] + '_0_' + str(j)
        data_left['answers'] = {'answer_start' : [answer_pos_left], 'text' : [answer_text_left]}
        return data_left

    def right_augmentation(self, data, j) :
        context = data['context']
        answer = data['answers']
        answer_pos, answer_text = answer['answer_start'][0], answer['text'][0]

        answer_pos_right = min(len(context)-1,answer_pos + len(answer_text) + np.random.randint(self.min_len, self.max_len))
        while context[answer_pos_right] != ' ' and answer_pos_right < len(context)-1:
            answer_pos_right += 1
        answer_text_right = context[answer_pos:answer_pos_right+1] 

        data_right = copy.deepcopy(data)
        data_right['id'] = data['id'] + '_1_' + str(j)
        data_right['answers'] = {'answer_start' : [answer_pos], 'text' : [answer_text_right]}
        return data_right

    def mid_augmentation(self, data, j) :   
        context = data['context']
        answer = data['answers']
        answer_pos = answer['answer_start'][0]

        answer_pos_left = max(0,answer_pos - np.random.randint(self.min_len/2, self.max_len/2))
        while context[answer_pos_left] != ' ' and answer_pos_left >= 0 :
            answer_pos_left -= 1
        answer_pos_left += 1

        answer_pos_right = min(len(context)-1,answer_pos + np.random.randint(self.min_len/2, self.max_len/2))
        while context[answer_pos_right] != ' ' and answer_pos_right < len(context)-1:
            answer_pos_right += 1

        answer_text = context[answer_pos_left:answer_pos_right+1] 
        data_mid = copy.deepcopy(data)
        data_mid['id'] = data['id'] + '_2_' + str(j)
        data_mid['answers'] = {'answer_start' : [answer_pos], 'text' : [answer_text]}
        return data_mid
